In [2]:
from mss import mss
import cv2
import pyautogui
from PIL import Image, ImageTk
import tkinter as tk
import time
import numpy as np

In [3]:
# Movements control:
steer_up_key = "W"
steer_down_key = "S"
steer_right_key = "R"
steer_left_key = "L"

view_up_key = "I"
view_down_key = "K"
view_left_key = "J"
view_right_key = "L"

def rotate_right_down():
    pyautogui.keyDown("L")
    
def rotate_right_up():
    pyautogui.keyUp("L")
    
def steer_up():
    pyautogui.keyDown("W")
    time.sleep(0.1)
    pyautogui.keyUp("W")

def steer_down():
    pyautogui.keyDown("S")
    time.sleep(0.1)
    pyautogui.keyUp("S")

In [4]:
# Pattern matcher for aim circle
def match_live_image(pattern, image_live):
    # Target is in gray already preprocessed beforehand
    pattern_cv2 = np.array(pattern)[:, :, ::-1].copy()
    tar_gray = cv2.cvtColor(pattern_cv2, cv2.COLOR_BGR2GRAY)


    # Convert to HSV Color Space:
    image_live_cv2 = np.array(image_live)[:, :, ::-1].copy()
    hsv_image = cv2.cvtColor(image_live_cv2, cv2.COLOR_BGR2HSV)


    # Define a Green Color Range:
    lower_green = np.array([50, 218, 183])  # Adjust these values based on your specific shade of green
    upper_green = np.array([175, 255, 255])
    mask = cv2.inRange(hsv_image, lower_green, upper_green)
    # Apply the Mask:
    green_areas = cv2.bitwise_and(image_live_cv2, image_live_cv2, mask=mask)
    
    # Grayscale and blur to remove noise:
    gray_image = cv2.cvtColor(green_areas, cv2.COLOR_BGR2GRAY)
    
    # Template Matching:
    result = cv2.matchTemplate(gray_image, tar_gray, cv2.TM_CCOEFF_NORMED)


    threshold = 0.27
    is_found = False
    loc = np.where(result >= threshold)
    
    # Now Aiming is complete fire down
    if(len(loc[0]) > 0):
        is_found = True
    
    
    for pt in zip(*loc[::-1]):
        cv2.rectangle(image_live_cv2, pt, (pt[0] + pattern_cv2.shape[1], pt[1] + pattern_cv2.shape[0]), (0, 0, 255), 2)
    
    image_live_cv2 = image_live_cv2[:, :, ::-1].copy()
    return image_live_cv2, is_found

In [1]:
# Real-time player
# x=81, y=32
# x=1343, y=744
region = {'left': 80, 'top': 32, 'width': 950, 'height': 531}
print("Starting in 5 seconds.")
time.sleep(5);
# Create a tkinter window
root = tk.Tk()
root.title("Replay")
initial_x = 81
initial_y = 536
root.geometry(f"+{initial_x}+{initial_y}")

# Why need label??
# Create a label to display the image in the tkinter window
label = tk.Label(root)
label.pack()

def update_image(label, img):
    image_tk = ImageTk.PhotoImage(img)
    label.configure(image=image_tk)
    label.image = image_tk

# Initialize
keep_playing = True;
def exit_loop(event):            
    global keep_playing
    keep_playing = False

root.bind('<`>', exit_loop)

# VARIABLES
pattern = Image.open("./images/aim_complete.png")
frame_delay = 0
wc1 = 8
weapon_cool_1 = wc1
is_notfound_cnt = 0
is_rotating_right = False
steer_vert = 0

def update_variables(time):
    global weapon_cool_1
    weapon_cool_1 = max(0, weapon_cool_1 - time)

# Image as video
while keep_playing:
    with mss() as sct:
        screenshot = sct.grab(region)
        cur_img = Image.frombytes('RGB', screenshot.size, screenshot.rgb)
    
    cur_img_np, is_found = match_live_image(pattern, cur_img)
    cur_img_pil = Image.fromarray(cur_img_np)
    
    if is_found:
        is_notfound_cnt = 0
        if is_rotating_right == True:
            is_rotating_right = False
            rotate_right_up()
            
        if steer_vert > 0:
            steer_down()
            steer_vert -= 1
            
        if weapon_cool_1 <= 0:
            weapon_cool_1 = wc1
            pyautogui.keyDown("Num1")
            time.sleep(0.1)
            pyautogui.keyUp("Num1")
            
    elif is_notfound_cnt > 20:
        is_notfound_cnt = 5
        if steer_vert < 2:
            steer_vert += 1
            steer_up()
            
        if is_rotating_right == False:
            is_rotating_right = True
            rotate_right_down()
    else:
        is_notfound_cnt += 1
        
    
    update_image(label, cur_img_pil)
    root.update()
    update_variables(frame_delay+0.1)
    time.sleep(frame_delay)

root.destroy()